In [1]:
import cv2
import numpy as np
from keras.models import model_from_json

2024-10-12 14:58:18.367492: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-12 14:58:18.606347: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-12 14:58:19.940296: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
tag_dict = {0:"Angry", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Neural", 5: "Sad", 6:"Surprised"}

json_file = open("/mnt/d/repositories/emotion_detection/models/best_emotion_model_structure_epoch_4.json", "r")
json_loaded_model = json_file.read()
json_file.close()
emotion_model = model_from_json(json_loaded_model)

emotion_model.load_weights("/mnt/d/repositories/emotion_detection/models/best_emotion_model_epoch_4.h5")
print("model loaded")

2024-10-12 15:01:46.824003: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-10-12 15:01:47.208849: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-10-12 15:01:47.208951: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-10-12 15:01:47.212669: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-10-12 15:01:47.212742: I tensorflow/compile

model loaded


In [ ]:
import cv2
import numpy as np

# Function to preprocess the face (for emotion prediction)
def preprocess_face(face):
    # Resize the face to the size expected by your model (e.g., 48x48)
    face = cv2.resize(face, (48, 48))
    # Normalize the face to [0, 1]
    face = face / 255.0
    # Expand dimensions to match the input shape (1, height, width, channels)
    face = np.expand_dims(face, axis=0)  # Add batch dimension
    face = np.expand_dims(face, axis=-1) # Add channel dimension for grayscale
    return face

# Function to predict emotion on a given face
def predict_emotion_on_face(face):
    # Preprocess the face
    preprocessed_face = preprocess_face(face)
    
    # Predict the emotion
    predictions = emotion_model.predict(preprocessed_face)
    predicted_class = np.argmax(predictions, axis=1)[0]
    confidence = predictions[0][predicted_class]
    
    return tag_dict[predicted_class], confidence

# Function to detect faces, draw bounding boxes, and predict emotions
def detect_and_predict_emotion(image_path):
    # Load the image
    img = cv2.imread(image_path)
    # Convert the image to grayscale (needed for face detection)
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Load the Haar Cascade for face detection
    face_cascade = cv2.CascadeClassifier("/mnt/d/repositories/emotion_detection/models/haarcascade_frontalface_default.xml")

    # Detect faces in the image
    # Detect faces in the image
    faces = face_cascade.detectMultiScale(gray_img, scaleFactor=1.1, minNeighbors=3)


    # If no faces are detected, return
    if len(faces) == 0:
        print("No faces detected.")
        return

    # Iterate over the detected faces
    for (x, y, w, h) in faces:
        # Extract the region of interest (the face)
        face = gray_img[y:y+h, x:x+w]

        # Predict the emotion for the detected face
        predicted_emotion, confidence = predict_emotion_on_face(face)

        # Draw a bounding box around the face
        cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)

        # Display the predicted emotion and confidence on the image
        if confidence > 0.5:  # Display only if confidence is high
            label = f"{predicted_emotion} ({confidence*100:.2f}%)"
            cv2.putText(img, label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

    # Show the final image with bounding boxes and predictions
    cv2.imshow("Emotion Detection", img)
    cv2.waitKey(0)  # Wait until a key is pressed
    cv2.destroyAllWindows()

# Test the model on a new image
image_path = '/mnt/d/repositories/emotion_detection/data/raw/test/fear/PrivateTest_518212.jpg'  # Change this to your test image path
detect_and_predict_emotion(image_path)



1/1 [==============================] - 0s 146ms/step
